##In-Depth Look into Convolutional Neural Networks



In [ ]:
# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, LocallyConnected1D, InputLayer, Flatten, LocallyConnected2D, Conv2D, Dropout
from keras.optimizers import SGD
from keras import utils as np_utils

In [ ]:
img_h = img_w = 16             
img_size_flat = img_h * img_w  # 16x16=256, the total number of pixels per image
n_classes = 10                 # total classes

In [ ]:
train_set = np.genfromtxt("./mnist_train.txt")
test_set = np.genfromtxt("./mnist_test.txt")

In [ ]:
print("training dataset shape: ", train_set.shape)
print("test dataset shape: ", test_set.shape)

training dataset shape:  (7291, 257)
test dataset shape:  (2007, 257)


In [ ]:
# extract labels and data 
train_labels = train_set[:,0]
test_labels = test_set[:,0]
print("training labels shape: ", train_labels.shape)
print("testing labels shape: ", test_labels.shape)

train_data = train_set[:,1:]
test_data = test_set[:,1:]
print("training data shape: ", train_data.shape)
print("testing data shape: ", test_data.shape)

training labels shape:  (7291,)
testing labels shape:  (2007,)
training data shape:  (7291, 256)
testing data shape:  (2007, 256)


In [ ]:
# reshape train and test datasets
train_data = train_data.reshape((train_data.shape[0], img_h, img_w, 1))
test_data = test_data.reshape((test_data.shape[0], img_h, img_w, 1))
print("reshaped training data: ", train_data.shape)
print("reshaped testing data: ", test_data.shape)

# use one hot encoding for outputs
train_labels = np_utils.to_categorical(train_labels)
test_labels = np_utils.to_categorical(test_labels)
print("one hot encoded training labels shape: ", train_labels.shape)
print("one hot encoded testing labels shape: ", test_labels.shape)

reshaped training data:  (7291, 16, 16, 1)
reshaped testing data:  (2007, 16, 16, 1)
one hot encoded training labels shape:  (7291, 10)
one hot encoded testing labels shape:  (2007, 10)


###Task 1: Neural Network Design

In [ ]:
def define_fully_connected_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Flatten())
  model.add(Dense(24, activation='relu', kernel_initializer=initializer))
  model.add(Dense(128, activation='relu', kernel_initializer=initializer))
  model.add(Dense(256, activation='tanh', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

The above model is a fully connected model with 3 hidden layers. The learning rate, momentum, image dimensions and initalizer choise are passed in as hyperparameters. The hidden layers use the relu, relu and tanh activation functions respectively. The output of the network has ten units, one for each target class. The target labels are passed as one hot encoded vectors. 

In [ ]:
def locally_connected_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(LocallyConnected2D(64, (3,3), activation="relu", kernel_initializer=initializer)) 
  model.add(LocallyConnected2D(32, (3,3), activation="relu", kernel_initializer=initializer))
  model.add(LocallyConnected2D(32, (3,3), activation="tanh", kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

The above model is a locally connected model with 3 hidden locally connected layers. There is one hidden dense layer with 64 units. The learning rate, momentum, image dimensions and initalizer choise are passed in as hyperparameters. The hidden layers use the relu, relu, tanh and relu activation functions respectively. In this locally connected architecture, the filters used during convolution are only passed over a local portion of the image the weights are not shared with the entire model but only the direct local neighbors. The output of the network has ten units, one for each target class. The target labels are passed as one hot encoded vectors. 

In [ ]:
def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

The above model is a convolutional network with 2D 3x3 kernels that will be convolved across the whole image. This network has 3 convolutional layers and 1 hidden dense layer. The output layer has 10 units, one for each target class.

###Task 2: Techniques for Optimization

#### Experiment 1: Parameter Initialization

In [ ]:
# Experiment 1 fixed hyper-params
epochs = 15
batch_size = 64
lr = 0.01
momentum = 0.9

###### Model 1: Fully Connected Model

The three initializers used to set the model weights are 'Zeros', 'HeUniform' and 'TruncatedNormal'. The comparison of training loss(*loss*), training accuracy (*accuracy*), validation loss(*val_loss*) and validation accuracy(*val_accuracy*) for each experiment with varying initializers can be seen in the training process below. The ranking of initializers best to worst (when analyzing training loss and accuracy) is: 1) HeUniform 2)TruncatedNormal 3)Zeros.

In [ ]:
# define initializer --> slow loss convergence and poor accuracy
initializer = 'Zeros'
fc_model = define_fully_connected_model(lr, momentum, img_h, img_w, initializer)
fc_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))


Epoch 1/15
114/114 [==============================] - 1s 5ms/step - loss: 2.2855 - accuracy: 0.1548 - val_loss: 2.2670 - val_accuracy: 0.1789
Epoch 2/15
114/114 [==============================] - 0s 3ms/step - loss: 2.2719 - accuracy: 0.1613 - val_loss: 2.2657 - val_accuracy: 0.1789
Epoch 3/15
114/114 [==============================] - 0s 3ms/step - loss: 2.2721 - accuracy: 0.1585 - val_loss: 2.2647 - val_accuracy: 0.1789
Epoch 4/15
114/114 [==============================] - 0s 3ms/step - loss: 2.2762 - accuracy: 0.1628 - val_loss: 2.2672 - val_accuracy: 0.1789
Epoch 5/15
114/114 [==============================] - 0s 3ms/step - loss: 2.2692 - accuracy: 0.1636 - val_loss: 2.2656 - val_accuracy: 0.1789
Epoch 6/15
114/114 [==============================] - 0s 3ms/step - loss: 2.2716 - accuracy: 0.1650 - val_loss: 2.2665 - val_accuracy: 0.1789
Epoch 7/15
114/114 [==============================] - 0s 3ms/step - loss: 2.2794 - accuracy: 0.1586 - val_loss: 2.2674 - val_accuracy: 0.1789
Epoch 

In [ ]:
# define initializer --> fast loss convergence and good accuracy
initializer = 'HeUniform'
fc_model = define_fully_connected_model(lr, momentum, img_h, img_w, initializer)
fc_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 1s 4ms/step - loss: 1.1294 - accuracy: 0.6288 - val_loss: 0.4106 - val_accuracy: 0.8809
Epoch 2/15
114/114 [==============================] - 0s 3ms/step - loss: 0.2099 - accuracy: 0.9406 - val_loss: 0.3448 - val_accuracy: 0.9058
Epoch 3/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1418 - accuracy: 0.9557 - val_loss: 0.3267 - val_accuracy: 0.9118
Epoch 4/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1148 - accuracy: 0.9699 - val_loss: 0.3246 - val_accuracy: 0.9123
Epoch 5/15
114/114 [==============================] - 0s 3ms/step - loss: 0.0903 - accuracy: 0.9711 - val_loss: 0.3154 - val_accuracy: 0.9198
Epoch 6/15
114/114 [==============================] - 0s 3ms/step - loss: 0.0725 - accuracy: 0.9800 - val_loss: 0.3135 - val_accuracy: 0.9198
Epoch 7/15
114/114 [==============================] - 0s 3ms/step - loss: 0.0547 - accuracy: 0.9862 - val_loss: 0.3116 - val_accuracy: 0.9248
Epoch 

In [ ]:
# define initializer --> fast loss convergence and good(but worse than previous case) accuracy
initializer = 'TruncatedNormal'
fc_model = define_fully_connected_model(lr, momentum, img_h, img_w, initializer)
fc_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 1s 4ms/step - loss: 1.9825 - accuracy: 0.2999 - val_loss: 0.5571 - val_accuracy: 0.8306
Epoch 2/15
114/114 [==============================] - 0s 3ms/step - loss: 0.3504 - accuracy: 0.8962 - val_loss: 0.4162 - val_accuracy: 0.8899
Epoch 3/15
114/114 [==============================] - 0s 3ms/step - loss: 0.2341 - accuracy: 0.9294 - val_loss: 0.3799 - val_accuracy: 0.8979
Epoch 4/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1829 - accuracy: 0.9464 - val_loss: 0.3487 - val_accuracy: 0.9008
Epoch 5/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1544 - accuracy: 0.9524 - val_loss: 0.3426 - val_accuracy: 0.9128
Epoch 6/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1436 - accuracy: 0.9597 - val_loss: 0.3205 - val_accuracy: 0.9133
Epoch 7/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1084 - accuracy: 0.9651 - val_loss: 0.3209 - val_accuracy: 0.9128
Epoch 

###### Model 2: Locally Connected Model

The three initializers used to set the model weights are 'Zeros', 'RandomNormal' and 'HeUniform'. The comparison of training loss(*loss*), training accuracy (*accuracy*), validation loss(*val_loss*) and validation accuracy(*val_accuracy*) for each experiment with varying initializers can be seen in the training process below. The ranking of initializers best to worst (when analyzing training loss and accuracy) is: 1) RandomNormal 2)HeUniform 3)Zeros.

In [ ]:
# define initializer --> slow loss convergence and poor accuracy
initializer = 'Zeros'
loc_con_model = locally_connected_model(lr, momentum, img_h, img_w, initializer)
loc_con_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 9s 30ms/step - loss: 2.2924 - accuracy: 0.1596 - val_loss: 2.2677 - val_accuracy: 0.1789
Epoch 2/15
114/114 [==============================] - 1s 12ms/step - loss: 2.2738 - accuracy: 0.1623 - val_loss: 2.2648 - val_accuracy: 0.1789
Epoch 3/15
114/114 [==============================] - 1s 13ms/step - loss: 2.2679 - accuracy: 0.1671 - val_loss: 2.2651 - val_accuracy: 0.1789
Epoch 4/15
114/114 [==============================] - 1s 12ms/step - loss: 2.2679 - accuracy: 0.1635 - val_loss: 2.2652 - val_accuracy: 0.1789
Epoch 5/15
114/114 [==============================] - 1s 13ms/step - loss: 2.2691 - accuracy: 0.1652 - val_loss: 2.2655 - val_accuracy: 0.1789
Epoch 6/15
114/114 [==============================] - 1s 12ms/step - loss: 2.2729 - accuracy: 0.1618 - val_loss: 2.2654 - val_accuracy: 0.1789
Epoch 7/15
114/114 [==============================] - 1s 13ms/step - loss: 2.2678 - accuracy: 0.1619 - val_loss: 2.2654 - val_accuracy: 0.1789

In [ ]:
# define initializer --> fast loss convergence and good accuracy
initializer = 'RandomNormal'
loc_con_model = locally_connected_model(lr, momentum, img_h, img_w, initializer)
loc_con_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 9s 29ms/step - loss: 1.5699 - accuracy: 0.5130 - val_loss: 0.4756 - val_accuracy: 0.8520
Epoch 2/15
114/114 [==============================] - 1s 12ms/step - loss: 0.3057 - accuracy: 0.9131 - val_loss: 0.3908 - val_accuracy: 0.8914
Epoch 3/15
114/114 [==============================] - 1s 12ms/step - loss: 0.1964 - accuracy: 0.9473 - val_loss: 0.3034 - val_accuracy: 0.9128
Epoch 4/15
114/114 [==============================] - 1s 12ms/step - loss: 0.1544 - accuracy: 0.9593 - val_loss: 0.2744 - val_accuracy: 0.9228
Epoch 5/15
114/114 [==============================] - 1s 12ms/step - loss: 0.1215 - accuracy: 0.9704 - val_loss: 0.2497 - val_accuracy: 0.9347
Epoch 6/15
114/114 [==============================] - 1s 12ms/step - loss: 0.0864 - accuracy: 0.9796 - val_loss: 0.2364 - val_accuracy: 0.9377
Epoch 7/15
114/114 [==============================] - 1s 12ms/step - loss: 0.0774 - accuracy: 0.9785 - val_loss: 0.2298 - val_accuracy: 0.9377

In [ ]:
# define initializer --> fast loss convergence and good(but worse than previous case) accuracy
initializer = 'HeUniform'
loc_con_model = locally_connected_model(lr, momentum, img_h, img_w, initializer)
loc_con_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 9s 29ms/step - loss: 2.2793 - accuracy: 0.1602 - val_loss: 2.2543 - val_accuracy: 0.1789
Epoch 2/15
114/114 [==============================] - 1s 12ms/step - loss: 2.2486 - accuracy: 0.1637 - val_loss: 2.1954 - val_accuracy: 0.3074
Epoch 3/15
114/114 [==============================] - 1s 12ms/step - loss: 2.1190 - accuracy: 0.3062 - val_loss: 1.6247 - val_accuracy: 0.3837
Epoch 4/15
114/114 [==============================] - 1s 13ms/step - loss: 1.3544 - accuracy: 0.5207 - val_loss: 0.8435 - val_accuracy: 0.7309
Epoch 5/15
114/114 [==============================] - 1s 13ms/step - loss: 0.6206 - accuracy: 0.8051 - val_loss: 0.6225 - val_accuracy: 0.8042
Epoch 6/15
114/114 [==============================] - 1s 13ms/step - loss: 0.4473 - accuracy: 0.8550 - val_loss: 0.5719 - val_accuracy: 0.8311
Epoch 7/15
114/114 [==============================] - 1s 12ms/step - loss: 0.3982 - accuracy: 0.8724 - val_loss: 0.5352 - val_accuracy: 0.8331

###### Model 3: Convolutional Model

The three initializers used to set the model weights are 'Zeros', 'HeUniform' and 'HeNormal'. The comparison of training loss(*loss*), training accuracy (*accuracy*), validation loss(*val_loss*) and validation accuracy(*val_accuracy*) for each experiment with varying initializers can be seen in the training process below. The ranking of initializers best to worst (when analyzing training loss and accuracy) is: 1) HeUniform 2)HeRandom 3)Zeros.

In [ ]:
# define initializer --> slow loss convergence and poor accuracy
initializer = 'Zeros'
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 2s 6ms/step - loss: 2.2927 - accuracy: 0.1554 - val_loss: 2.2672 - val_accuracy: 0.1789
Epoch 2/15
114/114 [==============================] - 0s 4ms/step - loss: 2.2729 - accuracy: 0.1611 - val_loss: 2.2653 - val_accuracy: 0.1789
Epoch 3/15
114/114 [==============================] - 0s 4ms/step - loss: 2.2698 - accuracy: 0.1636 - val_loss: 2.2651 - val_accuracy: 0.1789
Epoch 4/15
114/114 [==============================] - 0s 4ms/step - loss: 2.2674 - accuracy: 0.1675 - val_loss: 2.2654 - val_accuracy: 0.1789
Epoch 5/15
114/114 [==============================] - 0s 4ms/step - loss: 2.2726 - accuracy: 0.1573 - val_loss: 2.2651 - val_accuracy: 0.1789
Epoch 6/15
114/114 [==============================] - 0s 4ms/step - loss: 2.2688 - accuracy: 0.1590 - val_loss: 2.2648 - val_accuracy: 0.1789
Epoch 7/15
114/114 [==============================] - 0s 4ms/step - loss: 2.2661 - accuracy: 0.1680 - val_loss: 2.2655 - val_accuracy: 0.1789
Epoch 

In [ ]:
# redefine model to avoid errors

def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# define initializer --> fast loss convergence and good accuracy
initializer = 'HeUniform'
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 1s 5ms/step - loss: 0.8805 - accuracy: 0.7141 - val_loss: 0.2922 - val_accuracy: 0.9173
Epoch 2/15
114/114 [==============================] - 0s 4ms/step - loss: 0.1107 - accuracy: 0.9684 - val_loss: 0.2476 - val_accuracy: 0.9367
Epoch 3/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0582 - accuracy: 0.9835 - val_loss: 0.2250 - val_accuracy: 0.9372
Epoch 4/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0314 - accuracy: 0.9918 - val_loss: 0.2329 - val_accuracy: 0.9502
Epoch 5/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0177 - accuracy: 0.9950 - val_loss: 0.2278 - val_accuracy: 0.9522
Epoch 6/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0158 - accuracy: 0.9963 - val_loss: 0.2445 - val_accuracy: 0.9527
Epoch 7/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0114 - accuracy: 0.9969 - val_loss: 0.2319 - val_accuracy: 0.9562
Epoch 

In [ ]:
# redefine model to avoid errors

def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# define initializer --> fast loss convergence and good(but worse than previous case) accuracy
initializer = 'HeNormal'
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 1s 5ms/step - loss: 0.7936 - accuracy: 0.7504 - val_loss: 0.2472 - val_accuracy: 0.9357
Epoch 2/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0971 - accuracy: 0.9748 - val_loss: 0.2187 - val_accuracy: 0.9457
Epoch 3/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0446 - accuracy: 0.9867 - val_loss: 0.2220 - val_accuracy: 0.9407
Epoch 4/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0351 - accuracy: 0.9904 - val_loss: 0.2223 - val_accuracy: 0.9457
Epoch 5/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0168 - accuracy: 0.9961 - val_loss: 0.2716 - val_accuracy: 0.9497
Epoch 6/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0122 - accuracy: 0.9975 - val_loss: 0.2536 - val_accuracy: 0.9517
Epoch 7/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0060 - accuracy: 0.9980 - val_loss: 0.2532 - val_accuracy: 0.9557
Epoch 

In [ ]:
# redefine model to avoid errors

def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#### Experiment 2: Varying Learning Rates

In [ ]:
# Experiment 2 fixed hyper-params
epochs = 15
batch_size = 64
# lr = 0.01
initializer = 'HeUniform'
momentum = 0.9

###### Model 1: Fully Connected Model

In [ ]:
# define initializer --> slow loss convergence and poor accuracy
lr = 1e-6
fc_model = define_fully_connected_model(lr, momentum, img_h, img_w, initializer)
fc_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))


Epoch 1/15
114/114 [==============================] - 1s 4ms/step - loss: 2.4262 - accuracy: 0.2134 - val_loss: 2.4463 - val_accuracy: 0.1978
Epoch 2/15
114/114 [==============================] - 0s 3ms/step - loss: 2.4115 - accuracy: 0.2195 - val_loss: 2.4348 - val_accuracy: 0.1978
Epoch 3/15
114/114 [==============================] - 0s 3ms/step - loss: 2.4113 - accuracy: 0.2147 - val_loss: 2.4238 - val_accuracy: 0.1988
Epoch 4/15
114/114 [==============================] - 0s 3ms/step - loss: 2.4059 - accuracy: 0.2106 - val_loss: 2.4131 - val_accuracy: 0.2033
Epoch 5/15
114/114 [==============================] - 0s 3ms/step - loss: 2.3890 - accuracy: 0.2119 - val_loss: 2.4028 - val_accuracy: 0.2043
Epoch 6/15
114/114 [==============================] - 0s 3ms/step - loss: 2.3885 - accuracy: 0.2118 - val_loss: 2.3929 - val_accuracy: 0.2053
Epoch 7/15
114/114 [==============================] - 0s 3ms/step - loss: 2.3713 - accuracy: 0.2167 - val_loss: 2.3833 - val_accuracy: 0.2048
Epoch 

In [ ]:
# define initializer --> fast loss convergence and good accuracy
lr = 0.01
fc_model = define_fully_connected_model(lr, momentum, img_h, img_w, initializer)
fc_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 1s 4ms/step - loss: 1.0615 - accuracy: 0.6543 - val_loss: 0.3827 - val_accuracy: 0.8859
Epoch 2/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1960 - accuracy: 0.9392 - val_loss: 0.3339 - val_accuracy: 0.9098
Epoch 3/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1283 - accuracy: 0.9602 - val_loss: 0.3038 - val_accuracy: 0.9128
Epoch 4/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1052 - accuracy: 0.9694 - val_loss: 0.2838 - val_accuracy: 0.9243
Epoch 5/15
114/114 [==============================] - 0s 3ms/step - loss: 0.0779 - accuracy: 0.9780 - val_loss: 0.2963 - val_accuracy: 0.9213
Epoch 6/15
114/114 [==============================] - 0s 3ms/step - loss: 0.0665 - accuracy: 0.9799 - val_loss: 0.2786 - val_accuracy: 0.9268
Epoch 7/15
114/114 [==============================] - 0s 3ms/step - loss: 0.0574 - accuracy: 0.9840 - val_loss: 0.2793 - val_accuracy: 0.9253
Epoch 

In [ ]:
# define initializer --> fast loss convergence and good(but worse than previous case) accuracy
lr = 0.09
fc_model = define_fully_connected_model(lr, momentum, img_h, img_w, initializer)
fc_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 1s 4ms/step - loss: 1.0767 - accuracy: 0.6814 - val_loss: 0.6738 - val_accuracy: 0.8600
Epoch 2/15
114/114 [==============================] - 0s 3ms/step - loss: 0.3588 - accuracy: 0.9137 - val_loss: 0.5307 - val_accuracy: 0.8824
Epoch 3/15
114/114 [==============================] - 0s 3ms/step - loss: 0.2505 - accuracy: 0.9377 - val_loss: 0.4687 - val_accuracy: 0.9058
Epoch 4/15
114/114 [==============================] - 0s 3ms/step - loss: 0.2002 - accuracy: 0.9513 - val_loss: 0.5214 - val_accuracy: 0.8939
Epoch 5/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1882 - accuracy: 0.9505 - val_loss: 0.5208 - val_accuracy: 0.9008
Epoch 6/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1751 - accuracy: 0.9560 - val_loss: 0.5490 - val_accuracy: 0.9063
Epoch 7/15
114/114 [==============================] - 0s 3ms/step - loss: 0.1517 - accuracy: 0.9625 - val_loss: 0.5361 - val_accuracy: 0.9113
Epoch 

###### Model 2: Locally Connected Model

In [ ]:
# set best initializer for Model 2
initializer = "RandomNormal"

In [ ]:
# define initializer --> slow loss convergence and poor accuracy
lr = 1e-6
loc_con_model = locally_connected_model(lr, momentum, img_h, img_w, initializer)
loc_con_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 9s 29ms/step - loss: 2.2945 - accuracy: 0.1108 - val_loss: 2.2858 - val_accuracy: 0.1181
Epoch 2/15
114/114 [==============================] - 1s 13ms/step - loss: 2.2914 - accuracy: 0.1172 - val_loss: 2.2846 - val_accuracy: 0.1236
Epoch 3/15
114/114 [==============================] - 1s 13ms/step - loss: 2.2924 - accuracy: 0.1088 - val_loss: 2.2834 - val_accuracy: 0.1271
Epoch 4/15
114/114 [==============================] - 1s 13ms/step - loss: 2.2876 - accuracy: 0.1291 - val_loss: 2.2822 - val_accuracy: 0.1325
Epoch 5/15
114/114 [==============================] - 1s 13ms/step - loss: 2.2871 - accuracy: 0.1230 - val_loss: 2.2810 - val_accuracy: 0.1375
Epoch 6/15
114/114 [==============================] - 1s 13ms/step - loss: 2.2865 - accuracy: 0.1325 - val_loss: 2.2798 - val_accuracy: 0.1435
Epoch 7/15
114/114 [==============================] - 1s 13ms/step - loss: 2.2844 - accuracy: 0.1412 - val_loss: 2.2787 - val_accuracy: 0.1485

In [ ]:
# define initializer --> fast loss convergence and good accuracy
lr = 0.01
loc_con_model = locally_connected_model(lr, momentum, img_h, img_w, initializer)
loc_con_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 9s 29ms/step - loss: 1.5511 - accuracy: 0.5068 - val_loss: 0.4766 - val_accuracy: 0.8510
Epoch 2/15
114/114 [==============================] - 1s 13ms/step - loss: 0.3144 - accuracy: 0.9102 - val_loss: 0.3688 - val_accuracy: 0.8864
Epoch 3/15
114/114 [==============================] - 1s 13ms/step - loss: 0.1984 - accuracy: 0.9493 - val_loss: 0.3030 - val_accuracy: 0.9218
Epoch 4/15
114/114 [==============================] - 1s 12ms/step - loss: 0.1484 - accuracy: 0.9626 - val_loss: 0.2828 - val_accuracy: 0.9193
Epoch 5/15
114/114 [==============================] - 1s 13ms/step - loss: 0.1139 - accuracy: 0.9734 - val_loss: 0.2724 - val_accuracy: 0.9287
Epoch 6/15
114/114 [==============================] - 1s 13ms/step - loss: 0.0965 - accuracy: 0.9748 - val_loss: 0.2392 - val_accuracy: 0.9297
Epoch 7/15
114/114 [==============================] - 1s 12ms/step - loss: 0.0695 - accuracy: 0.9821 - val_loss: 0.2304 - val_accuracy: 0.9362

In [ ]:
# define initializer --> fast loss convergence and good(but worse than previous case) accuracy
lr = 0.20
loc_con_model = locally_connected_model(lr, momentum, img_h, img_w, initializer)
loc_con_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 9s 29ms/step - loss: 0.8583 - accuracy: 0.7205 - val_loss: 0.5070 - val_accuracy: 0.8779
Epoch 2/15
114/114 [==============================] - 1s 13ms/step - loss: 0.3453 - accuracy: 0.9088 - val_loss: 0.3533 - val_accuracy: 0.9163
Epoch 3/15
114/114 [==============================] - 1s 13ms/step - loss: 0.1621 - accuracy: 0.9586 - val_loss: 0.5756 - val_accuracy: 0.8869
Epoch 4/15
114/114 [==============================] - 1s 13ms/step - loss: 0.3611 - accuracy: 0.9165 - val_loss: 1.3459 - val_accuracy: 0.7240
Epoch 5/15
114/114 [==============================] - 1s 13ms/step - loss: 1.1659 - accuracy: 0.6753 - val_loss: 1.9849 - val_accuracy: 0.3059
Epoch 6/15
114/114 [==============================] - 1s 13ms/step - loss: 2.0065 - accuracy: 0.2961 - val_loss: 2.2690 - val_accuracy: 0.1789
Epoch 7/15
114/114 [==============================] - 1s 13ms/step - loss: 2.2842 - accuracy: 0.1572 - val_loss: 2.2746 - val_accuracy: 0.1315

###### Model 3: Convolutional Model

In [ ]:
# set best initializer for Model 2
initializer = "HeUniform"

In [ ]:
# define initializer --> slow loss convergence and poor accuracy
lr = 1e-6
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 1s 5ms/step - loss: 2.6869 - accuracy: 0.0572 - val_loss: 2.6282 - val_accuracy: 0.0548
Epoch 2/15
114/114 [==============================] - 0s 4ms/step - loss: 2.5937 - accuracy: 0.0643 - val_loss: 2.5596 - val_accuracy: 0.0638
Epoch 3/15
114/114 [==============================] - 0s 4ms/step - loss: 2.5285 - accuracy: 0.0697 - val_loss: 2.5031 - val_accuracy: 0.0698
Epoch 4/15
114/114 [==============================] - 0s 4ms/step - loss: 2.4735 - accuracy: 0.0843 - val_loss: 2.4568 - val_accuracy: 0.0797
Epoch 5/15
114/114 [==============================] - 0s 4ms/step - loss: 2.4280 - accuracy: 0.0895 - val_loss: 2.4174 - val_accuracy: 0.1031
Epoch 6/15
114/114 [==============================] - 0s 4ms/step - loss: 2.3932 - accuracy: 0.1158 - val_loss: 2.3834 - val_accuracy: 0.1236
Epoch 7/15
114/114 [==============================] - 0s 4ms/step - loss: 2.3410 - accuracy: 0.1381 - val_loss: 2.3534 - val_accuracy: 0.1574
Epoch 

In [ ]:
# redefine model to avoid errors

def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# define initializer --> fast loss convergence and good accuracy
lr = 0.01
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 1s 5ms/step - loss: 0.8058 - accuracy: 0.7326 - val_loss: 0.2505 - val_accuracy: 0.9352
Epoch 2/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0836 - accuracy: 0.9767 - val_loss: 0.2319 - val_accuracy: 0.9477
Epoch 3/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0500 - accuracy: 0.9857 - val_loss: 0.2175 - val_accuracy: 0.9432
Epoch 4/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0204 - accuracy: 0.9950 - val_loss: 0.2115 - val_accuracy: 0.9462
Epoch 5/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0144 - accuracy: 0.9972 - val_loss: 0.2202 - val_accuracy: 0.9581
Epoch 6/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0059 - accuracy: 0.9990 - val_loss: 0.2363 - val_accuracy: 0.9567
Epoch 7/15
114/114 [==============================] - 0s 4ms/step - loss: 0.0037 - accuracy: 0.9997 - val_loss: 0.2279 - val_accuracy: 0.9557
Epoch 

In [ ]:
# redefine model to avoid errors

def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# define initializer --> fast loss convergence and good(but worse than previous case) accuracy
lr = 0.092
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
114/114 [==============================] - 1s 5ms/step - loss: 1.9160 - accuracy: 0.5018 - val_loss: 0.6043 - val_accuracy: 0.8221
Epoch 2/15
114/114 [==============================] - 0s 4ms/step - loss: 0.3455 - accuracy: 0.9012 - val_loss: 0.6012 - val_accuracy: 0.8660
Epoch 3/15
114/114 [==============================] - 0s 4ms/step - loss: 0.3419 - accuracy: 0.8979 - val_loss: 0.4417 - val_accuracy: 0.8869
Epoch 4/15
114/114 [==============================] - 0s 4ms/step - loss: 0.1858 - accuracy: 0.9497 - val_loss: 0.4070 - val_accuracy: 0.9013
Epoch 5/15
114/114 [==============================] - 0s 4ms/step - loss: 0.1374 - accuracy: 0.9586 - val_loss: 0.3745 - val_accuracy: 0.9098
Epoch 6/15
114/114 [==============================] - 0s 4ms/step - loss: 0.1598 - accuracy: 0.9567 - val_loss: 0.4863 - val_accuracy: 0.9033
Epoch 7/15
114/114 [==============================] - 0s 4ms/step - loss: 0.1252 - accuracy: 0.9634 - val_loss: 0.4582 - val_accuracy: 0.9218
Epoch 

In [ ]:
# redefine model to avoid errors

def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#### Experiment 3: Effect of Batch Size on Batch Normalization for Convolutional Network

Increasing batch size brings the sampled values closer to trained population values. Increasing batch size improves batch normalization. With a batch size that is too small the network is chasing a moving target. This change in the distribution of inputs to layers in the network is referred to the technical name “internal covariate shift.”

In [ ]:
# Experiment 3 fixed hyper-params --> with best values for CNN
epochs = 15
lr = 0.01
initializer = 'HeUniform'
momentum = 0.9

In [ ]:
# ineffective batch size
batch_size = 1
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
7291/7291 [==============================] - 22s 3ms/step - loss: 2.3019 - accuracy: 0.1679 - val_loss: 2.2790 - val_accuracy: 0.1789
Epoch 2/15
7291/7291 [==============================] - 21s 3ms/step - loss: 2.2986 - accuracy: 0.1476 - val_loss: 2.2726 - val_accuracy: 0.1789
Epoch 3/15
7291/7291 [==============================] - 21s 3ms/step - loss: 2.2942 - accuracy: 0.1435 - val_loss: 2.2889 - val_accuracy: 0.1789
Epoch 4/15
7291/7291 [==============================] - 21s 3ms/step - loss: 2.2901 - accuracy: 0.1584 - val_loss: 2.2747 - val_accuracy: 0.1789
Epoch 5/15
7291/7291 [==============================] - 21s 3ms/step - loss: 2.2904 - accuracy: 0.1573 - val_loss: 2.3103 - val_accuracy: 0.0987
Epoch 6/15
7291/7291 [==============================] - 21s 3ms/step - loss: 2.2922 - accuracy: 0.1559 - val_loss: 2.2878 - val_accuracy: 0.1315
Epoch 7/15
7291/7291 [==============================] - 22s 3ms/step - loss: 2.2906 - accuracy: 0.1482 - val_loss: 2.2841 - val_ac

In [ ]:
# redefine model to avoid errors

def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# effective batch size
batch_size = 128
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
57/57 [==============================] - 1s 8ms/step - loss: 1.1888 - accuracy: 0.6158 - val_loss: 0.3067 - val_accuracy: 0.9173
Epoch 2/15
57/57 [==============================] - 0s 4ms/step - loss: 0.1194 - accuracy: 0.9649 - val_loss: 0.2057 - val_accuracy: 0.9427
Epoch 3/15
57/57 [==============================] - 0s 5ms/step - loss: 0.0627 - accuracy: 0.9842 - val_loss: 0.1991 - val_accuracy: 0.9497
Epoch 4/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0310 - accuracy: 0.9913 - val_loss: 0.2153 - val_accuracy: 0.9527
Epoch 5/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0246 - accuracy: 0.9935 - val_loss: 0.2144 - val_accuracy: 0.9542
Epoch 6/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0145 - accuracy: 0.9972 - val_loss: 0.2437 - val_accuracy: 0.9492
Epoch 7/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0135 - accuracy: 0.9975 - val_loss: 0.2337 - val_accuracy: 0.9552
Epoch 8/15
57/57 [==

#### Experiment 4: Effect of Momentum on Convolutional Network Training

Different momentum values are contrasted on the convolutional neural network. To exaggerate the differences the values 0.01, 0.50 and 0.99 are used. With a higher momentum, a marginally better performance was observed. 

In [ ]:
# Experiment 4 fixed hyper-params --> with best values for CNN
epochs = 15
lr = 0.01
batch_size = 128
initializer = 'HeUniform'

In [ ]:
# redefine model to avoid errors

def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# momentum value 1
momentum = 0.01
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
57/57 [==============================] - 1s 7ms/step - loss: 1.3454 - accuracy: 0.5740 - val_loss: 0.5564 - val_accuracy: 0.8236
Epoch 2/15
57/57 [==============================] - 0s 4ms/step - loss: 0.3314 - accuracy: 0.8925 - val_loss: 0.3810 - val_accuracy: 0.8874
Epoch 3/15
57/57 [==============================] - 0s 4ms/step - loss: 0.2365 - accuracy: 0.9321 - val_loss: 0.3410 - val_accuracy: 0.8994
Epoch 4/15
57/57 [==============================] - 0s 5ms/step - loss: 0.1829 - accuracy: 0.9467 - val_loss: 0.3093 - val_accuracy: 0.9068
Epoch 5/15
57/57 [==============================] - 0s 4ms/step - loss: 0.1528 - accuracy: 0.9552 - val_loss: 0.3121 - val_accuracy: 0.9138
Epoch 6/15
57/57 [==============================] - 0s 4ms/step - loss: 0.1209 - accuracy: 0.9657 - val_loss: 0.2606 - val_accuracy: 0.9278
Epoch 7/15
57/57 [==============================] - 0s 4ms/step - loss: 0.1089 - accuracy: 0.9733 - val_loss: 0.2547 - val_accuracy: 0.9287
Epoch 8/15
57/57 [==

In [ ]:
# redefine model to avoid errors

def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# momentum value 2
momentum = 0.50
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
57/57 [==============================] - 1s 7ms/step - loss: 1.1253 - accuracy: 0.6323 - val_loss: 0.4372 - val_accuracy: 0.8705
Epoch 2/15
57/57 [==============================] - 0s 4ms/step - loss: 0.2446 - accuracy: 0.9235 - val_loss: 0.2985 - val_accuracy: 0.9193
Epoch 3/15
57/57 [==============================] - 0s 4ms/step - loss: 0.1640 - accuracy: 0.9571 - val_loss: 0.2651 - val_accuracy: 0.9302
Epoch 4/15
57/57 [==============================] - 0s 4ms/step - loss: 0.1158 - accuracy: 0.9664 - val_loss: 0.2539 - val_accuracy: 0.9362
Epoch 5/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0855 - accuracy: 0.9750 - val_loss: 0.2422 - val_accuracy: 0.9402
Epoch 6/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0786 - accuracy: 0.9810 - val_loss: 0.2349 - val_accuracy: 0.9412
Epoch 7/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0580 - accuracy: 0.9829 - val_loss: 0.2493 - val_accuracy: 0.9407
Epoch 8/15
57/57 [==

In [ ]:
# redefine model to avoid errors

def conv_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=initializer))
  model.add(Flatten())
  model.add(Dense(64, activation='relu', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# momentum value 3
momentum = 0.99
conv_model = conv_model(lr, momentum, img_h, img_w, initializer)
conv_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
57/57 [==============================] - 1s 7ms/step - loss: 1.1538 - accuracy: 0.6352 - val_loss: 0.5026 - val_accuracy: 0.8889
Epoch 2/15
57/57 [==============================] - 0s 4ms/step - loss: 0.2453 - accuracy: 0.9310 - val_loss: 0.4205 - val_accuracy: 0.9048
Epoch 3/15
57/57 [==============================] - 0s 4ms/step - loss: 0.1492 - accuracy: 0.9610 - val_loss: 0.3386 - val_accuracy: 0.9218
Epoch 4/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0873 - accuracy: 0.9725 - val_loss: 0.3099 - val_accuracy: 0.9412
Epoch 5/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0517 - accuracy: 0.9840 - val_loss: 0.4511 - val_accuracy: 0.9427
Epoch 6/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0356 - accuracy: 0.9901 - val_loss: 0.5992 - val_accuracy: 0.9472
Epoch 7/15
57/57 [==============================] - 0s 4ms/step - loss: 0.0647 - accuracy: 0.9841 - val_loss: 0.5146 - val_accuracy: 0.9492
Epoch 8/15
57/57 [==

###Task 3: Techniques for Improving Generalization

#### Method 1: Ensemble Learning

In this framework, 20 weak learners are used and their learned values combined to make one strong learner. In this way the final strong model is resiliant and performs well. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

In [ ]:
ensemble_train_set = np.genfromtxt("./mnist_train.txt")
ensemble_test_set = np.genfromtxt("./mnist_test.txt")

# extract labels and data 
ensemble_train_labels = ensemble_train_set[:,0]
ensemble_test_labels = ensemble_test_set[:,0]
print("ensemble training labels shape: ", ensemble_train_labels.shape)
print("ensemble testing labels shape: ", ensemble_test_labels.shape)

ensemble_train_data = ensemble_train_set[:,1:]
ensemble_test_data = ensemble_test_set[:,1:]
print("ensemble training data shape: ", ensemble_train_data.shape)
print("ensemble testing data shape: ", ensemble_test_data.shape)

ensemble training labels shape:  (7291,)
ensemble testing labels shape:  (2007,)
ensemble training data shape:  (7291, 256)
ensemble testing data shape:  (2007, 256)


In [ ]:
# Bagging
bg = BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5, max_features=1.0, n_estimators=20)
bg.fit(ensemble_train_data, ensemble_train_labels)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [ ]:
# validate on train set
bg.score(ensemble_train_data,ensemble_train_labels)

0.9916335207790427

In [ ]:
# validate on test set
bg.score(ensemble_test_data,ensemble_test_labels)

0.8824115595416044

#### Method 2: Dropout

In [ ]:
# best hyperparam values for Fully Connected Network
epochs = 15
lr = 0.01
batch_size = 128
momentum = 0.9
initializer = 'HeUniform'

In [ ]:
# redefine fully connected network WITHOUT dropout 
# reduce number of neurons to reduce overfitting

def define_fully_connected_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Flatten())
  model.add(Dense(10, activation='relu', kernel_initializer=initializer))
  model.add(Dense(32, activation='tanh', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# test fully connected network without dropout
fc_model = define_fully_connected_model(lr, momentum, img_h, img_w, initializer)
fc_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
57/57 [==============================] - 1s 6ms/step - loss: 1.8592 - accuracy: 0.3310 - val_loss: 0.9589 - val_accuracy: 0.7050
Epoch 2/15
57/57 [==============================] - 0s 3ms/step - loss: 0.6954 - accuracy: 0.8053 - val_loss: 0.6158 - val_accuracy: 0.8196
Epoch 3/15
57/57 [==============================] - 0s 3ms/step - loss: 0.4032 - accuracy: 0.9005 - val_loss: 0.5165 - val_accuracy: 0.8450
Epoch 4/15
57/57 [==============================] - 0s 3ms/step - loss: 0.3289 - accuracy: 0.9169 - val_loss: 0.4739 - val_accuracy: 0.8585
Epoch 5/15
57/57 [==============================] - 0s 3ms/step - loss: 0.2647 - accuracy: 0.9342 - val_loss: 0.4496 - val_accuracy: 0.8690
Epoch 6/15
57/57 [==============================] - 0s 3ms/step - loss: 0.2437 - accuracy: 0.9365 - val_loss: 0.4350 - val_accuracy: 0.8799
Epoch 7/15
57/57 [==============================] - 0s 3ms/step - loss: 0.2211 - accuracy: 0.9416 - val_loss: 0.4251 - val_accuracy: 0.8834
Epoch 8/15
57/57 [==

Add Dropout Layers to Fully Connected Network

In [ ]:
# re-define fully connected network WITH dropout layer
# Effective case with dropout rate = 0.5

def define_fully_connected_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Flatten())
  model.add(Dense(10, activation='relu', kernel_initializer=initializer))
  model.add(Dropout(0.2)) ## added dropout layer ****
  model.add(Dense(32, activation='tanh', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# test fully connected network WITH dropout
# Effective case with dropout rate = 0.2
fc_model = define_fully_connected_model(lr, momentum, img_h, img_w, initializer)
fc_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
57/57 [==============================] - 1s 5ms/step - loss: 2.1013 - accuracy: 0.2905 - val_loss: 1.0663 - val_accuracy: 0.6951
Epoch 2/15
57/57 [==============================] - 0s 3ms/step - loss: 1.1088 - accuracy: 0.6438 - val_loss: 0.7413 - val_accuracy: 0.8171
Epoch 3/15
57/57 [==============================] - 0s 3ms/step - loss: 0.8812 - accuracy: 0.7159 - val_loss: 0.6289 - val_accuracy: 0.8416
Epoch 4/15
57/57 [==============================] - 0s 3ms/step - loss: 0.7732 - accuracy: 0.7579 - val_loss: 0.5809 - val_accuracy: 0.8520
Epoch 5/15
57/57 [==============================] - 0s 3ms/step - loss: 0.7060 - accuracy: 0.7827 - val_loss: 0.5424 - val_accuracy: 0.8620
Epoch 6/15
57/57 [==============================] - 0s 3ms/step - loss: 0.6513 - accuracy: 0.7980 - val_loss: 0.5196 - val_accuracy: 0.8575
Epoch 7/15
57/57 [==============================] - 0s 3ms/step - loss: 0.6345 - accuracy: 0.7993 - val_loss: 0.5003 - val_accuracy: 0.8680
Epoch 8/15
57/57 [==

In [ ]:
# define fully connected model WITH dropout layer
# Ineffective case with dropout rate = 0.80 --> TOO HIGH DROPOUT RATE!

def define_fully_connected_model(lr_val, momentum_val, img_h, img_w, initializer):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_h, img_w, 1)))
  model.add(Flatten())
  model.add(Dense(10, activation='relu', kernel_initializer=initializer))
  model.add(Dropout(0.80)) ## added dropout layer ****
  model.add(Dense(32, activation='tanh', kernel_initializer=initializer))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=lr_val, momentum=momentum_val)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# test FC model WITH high dropout 
# Ineffective case with dropout rate = 0.80 --> TOO HIGH DROPOUT RATE!
fc_model = define_fully_connected_model(lr, momentum, img_h, img_w, initializer)
fc_model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/15
57/57 [==============================] - 1s 5ms/step - loss: 2.4474 - accuracy: 0.1552 - val_loss: 1.9412 - val_accuracy: 0.3523
Epoch 2/15
57/57 [==============================] - 0s 3ms/step - loss: 2.1401 - accuracy: 0.2118 - val_loss: 1.6900 - val_accuracy: 0.4006
Epoch 3/15
57/57 [==============================] - 0s 3ms/step - loss: 2.0628 - accuracy: 0.2378 - val_loss: 1.5499 - val_accuracy: 0.4405
Epoch 4/15
57/57 [==============================] - 0s 3ms/step - loss: 2.0164 - accuracy: 0.2578 - val_loss: 1.3953 - val_accuracy: 0.4813
Epoch 5/15
57/57 [==============================] - 0s 3ms/step - loss: 1.9856 - accuracy: 0.2610 - val_loss: 1.2803 - val_accuracy: 0.5177
Epoch 6/15
57/57 [==============================] - 0s 3ms/step - loss: 1.9436 - accuracy: 0.2701 - val_loss: 1.2072 - val_accuracy: 0.5635
Epoch 7/15
57/57 [==============================] - 0s 3ms/step - loss: 1.9087 - accuracy: 0.2770 - val_loss: 1.1755 - val_accuracy: 0.5346
Epoch 8/15
57/57 [==